In [ ]:
20

In [ ]:
20

In [1]:
from federatedscope.register import register_data
from federatedscope.register import register_trainer
from federatedscope.register import register_metric
from federatedscope.register import register_model




### Register model

In [2]:
#from federatedscope.contrib.model.mnist_model import call_my_net
#register_model("mynet", call_my_net)

### Register trainer

In [3]:


from federatedscope.contrib.trainer.laplacian_trainer_with_domain_separation_with_summation_1MINE_VAE import call_laplacian_trainer

register_trainer('laplacian_trainer', call_laplacian_trainer)

/home/michael/Master-Thesis/CKIM_Competition/venv/lib/python3.9/site-packages/torch_geometric/graphgym/imports.py:14: UserWarning: Please install 'pytorch_lightning' for using the GraphGym experiment manager via 'pip install pytorch_lightning'
  warnings.warn("Please install 'pytorch_lightning' for using the GraphGym "
/home/michael/Master-Thesis/CKIM_Competition/venv/lib/python3.9/site-packages/torch_geometric/graphgym/logger.py:23: UserWarning: Please install 'pytorch_lightning' for using the GraphGym experiment manager via 'pip install pytorch_lightning'
  warnings.warn("Please install 'pytorch_lightning' for using the GraphGym "


Device: cuda


### Register metric

### Create the config

#### Set data, model, trainer and metric

In [4]:
from federatedscope.core.configs.config import global_cfg, CN
cfg = global_cfg.clone()

cfg.merge_from_file("scripts/B-FHTL_exp_scripts/Graph-DT/fed_dom_sep_MI.yaml")
cfg.data.save_dir = 'test_dir'
# cfg.data.type = 'cikm_cup'
#cfg.data.root = 'data'
#cfg.data.shuffle=True
#cfg.data.transform = [['ToTensor'], ['Normalize', {'mean': [0.], 'std': [1]}]]
#cfg.model.type = 'gin'
#cfg.model.out_channels = 10
#cfg.model.hidden = 64
#cfg.model.task='graph'
#cfg.model.dropout = 0.5
#cfg.personalization.local_param = ['encoder_atom', 'encoder', 'clf']#['node_encoder', 'clf']
#cfg.train.batch_or_epoch = "epoch"
cfg.trainer.type = 'laplacian_trainer'
cfg.data.batch_size = 64
# cfg.eval.metric = ['mymetric']
cfg.params = CN()
cfg.params.alpha=0.1
cfg.params.csd_importance= 1e2
cfg.params.diff_importance = 1
cfg.params.mine_lr = 0.01
cfg.params.eps=1e-20
cfg.params.p=0.
cfg.params.lam = 0.01


#### configure other options

In [5]:
#cfg.use_gpu = True 0.05 [0.02-0.03] 0.04-0.06
#cfg.best_res_update_round_wise_key = "test_loss"

#cfg.federate.mode = 'standalone'

cfg.federate.method = \
    f'Laplacian_MINE_VAE_test'

#cfg.federate.local_update_steps = 20000000
cfg.personalization.local_update_steps = 20000000
#cfg.finetune.local_update_steps = 20000000
#cfg.train.local_update_steps = 1

cfg.federate.total_round_num = 2000
cfg.federate.client_num = 2
cfg.early_stop.patience = 3000
#cfg.train.optimizer.lr = 0.001
#cfg.train.optimizer.weight_decay = 0.0005
#cfg.grad.grad_clip = 2.0
cfg.criterion.type = 'CrossEntropyLoss'
#cfg.seed = 123
cfg.eval.freq = 1
cfg.eval.metrics = ['imp_ratio']
cfg.eval.report = ['avg']
cfg.eval.best_res_update_round_wise_key = 'val_imp_ratio'
cfg.eval.count_flops = False


In [6]:
import torch
#torch.manual_seed(0)
#torch.backends.cudnn.benchmark = False
#torch.use_deterministic_algorithms(F/home/michael/Desktop/backup files/FederatedScope/data/CIKM22Competitionalse)
#import random
#random.seed(0)
#import numpy as np
#np.random.seed(0)

In [7]:
from yacs.config import CfgNode
client_cfg_file = "scripts/B-FHTL_exp_scripts/Graph-DT/cfg_per_client_theirs.yaml"# "scripts/B-FHTL_exp_scripts/Graph-DT/cfg_per_client_ours_lr_local_steps.yaml"
client_cfg = CfgNode.load_cfg(open(client_cfg_file,
                                       'r')) if client_cfg_file else None


### Start the FL prosess

In [8]:
from federatedscope.core.auxiliaries.data_builder import get_data
from federatedscope.core.auxiliaries.utils import setup_seed, update_logger
from federatedscope.core.fed_runner import FedRunner
from federatedscope.core.auxiliaries.worker_builder import get_server_cls, get_client_cls


2023-01-07 21:15:21,298 (trainer_builder:11)WARNING: No module named 'federatedscope.contrib.optimizer' in `federatedscope.contrib.trainer`, some modules are not available.


In [9]:
from federatedscope.contrib.workers.laplacian_with_domain_separation_MI_client import LaplacianDomainSeparationMIClient
from federatedscope.contrib.workers.laplacian_server_dom_sep import LaplacianServerDomSep

setup_seed(cfg.seed)
update_logger(cfg)
data, modified_cfg = get_data(cfg)
cfg.merge_from_other_cfg(modified_cfg)


2023-01-07 21:15:21,308 (utils:129)INFO: the current machine is at 127.0.1.1
2023-01-07 21:15:21,308 (utils:131)INFO: the current dir is /home/michael/Master-Thesis/CKIM_Competition
2023-01-07 21:15:21,309 (utils:132)INFO: the output dir is exp/test_dir/Laplacian_MINE_VAE_test_gin_on_graph-dt_lr0.1_lstep1_/sub_exp_20230107211521


In [10]:
from federatedscope.contrib.workers.laplacian_with_domain_separation_1MINE_VAE_client import LaplacianDomainSeparation1MINEClient
from federatedscope.contrib.workers.laplacian_server import LaplacianServer

Fed_runner = FedRunner(data=data,
                       server_class=LaplacianServer,
                       client_class=LaplacianDomainSeparation1MINEClient,
                       config=cfg.clone(),
                       client_config=client_cfg)
Fed_runner.run()

rho: 0.0
server params: 
encoder_atom.atom_embedding_list.0.weight


/home/michael/Master-Thesis/CKIM_Competition/venv/lib/python3.9/site-packages/torch/nn/init.py:405: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")
2023-01-07 21:15:23,942 (fed_runner:249)INFO: Server #0 has been set up ... 
2023-01-07 21:15:23,952 (config:261)INFO: the used configs are: 
asyn:
  min_received_num: 2
  min_received_rate: -1.0
  timeout: 0
  use: True
attack:
  alpha_TV: 0.001
  alpha_prop_loss: 0
  attack_method: 
  attacker_id: -1
  classifier_PIA: randomforest
  info_diff_type: l2
  inject_round: 0
  max_ite: 400
  reconstruct_lr: 0.01
  reconstruct_optim: Adam
  target_label_ind: -1
backend: torch
cfg_file: 
criterion:
  type: MSELoss
data:
  args: []
  batch_size: 64
  cSBM_phi: [0.5, 0.5, 0.5]
  consistent_label_distribution: False
  drop_last: False
  graphsaint:
    num_steps: 30
    walk_length: 2
  loader: 
  num_workers: 0
  pre_transform: []
  quadratic:
    dim: 1
    max_curv: 12.5
  

encoder_atom.atom_embedding_list.1.weight
encoder_atom.atom_embedding_list.2.weight
encoder_atom.atom_embedding_list.3.weight
encoder_atom.atom_embedding_list.4.weight
encoder_atom.atom_embedding_list.5.weight
encoder_atom.atom_embedding_list.6.weight
encoder_atom.atom_embedding_list.7.weight
encoder_atom.atom_embedding_list.8.weight
encoder_atom.atom_embedding_list.9.weight
encoder_atom.atom_embedding_list.10.weight
encoder_atom.atom_embedding_list.11.weight
encoder_atom.atom_embedding_list.12.weight
encoder_atom.atom_embedding_list.13.weight
encoder_atom.atom_embedding_list.14.weight
encoder_atom.atom_embedding_list.15.weight
encoder_atom.atom_embedding_list.16.weight
encoder_atom.atom_embedding_list.17.weight
encoder_atom.atom_embedding_list.18.weight
encoder_atom.atom_embedding_list.19.weight
encoder_atom.atom_embedding_list.20.weight
encoder_atom.atom_embedding_list.21.weight
encoder.weight
encoder.bias
local_gnn.convs.0.nn.linears.0.weight
local_gnn.convs.0.nn.linears.0.bias
loca

2023-01-07 21:15:26,243 (laplacian_with_domain_separation_1MINE_VAE_client:142)INFO: {'Role': 'Client #2', 'Round': 0, 'Results_raw': {'train_imp_ratio': -5061.381218, 'train_avg_loss': 1.367766, 'train_loss': 700.296185, 'train_total': 512}}


in train
mi_local_global: 0.0005114786326885223
mi_global_fixed: -0.000591576099395752
rec_loss: 1.0010262727737427
kld_loss: 2.4962124824523926
mi_local_global: 8.248910307884216e-05
mi_global_fixed: 0.0003572702407836914
rec_loss: 1.0039217472076416
kld_loss: 2.462754964828491
mi_local_global: 0.0004407241940498352
mi_global_fixed: 0.001030467450618744
rec_loss: 1.0005099773406982
kld_loss: 2.415041923522949
mi_local_global: 0.0006409473717212677
mi_global_fixed: 0.0014789439737796783
rec_loss: 1.0082205533981323
kld_loss: 2.373008966445923
mi_local_global: 0.0020277462899684906
mi_global_fixed: -0.0019655711948871613
rec_loss: 1.0053366422653198
kld_loss: 2.352874994277954
mi_local_global: 0.0016130730509757996
mi_global_fixed: 0.0033873654901981354
rec_loss: 0.997795581817627
kld_loss: 2.316397190093994
mi_local_global: 0.0008163563907146454
mi_global_fixed: 0.0010843202471733093
rec_loss: 1.0037333965301514
kld_loss: 2.279433250427246
mi_local_global: 0.0014475025236606598
mi_glob

...

2023-01-07 21:16:43,134 (client:410)INFO: {'Role': 'Client #2', 'Round': 17, 'Results_raw': {'test_imp_ratio': -530.224707, 'test_avg_loss': 0.16701, 'test_loss': 10.688612, 'test_total': 64, 'val_imp_ratio': -776.750249, 'val_avg_loss': 0.232339, 'val_loss': 14.637346, 'val_total': 63}}
2023-01-07 21:16:43,134 (monitor:513)INFO: current_best=-764.14814, should_save=False


in val or test
mi_local_global: 0.000901736319065094
mi_global_fixed: -0.003277023322880268
rec_loss: 0.9064674377441406
kld_loss: 0.04873625934123993
in val or test
mi_local_global: 0.0014663077890872955
mi_global_fixed: -0.0032670795917510986
rec_loss: 0.9348123073577881
kld_loss: 0.051315467804670334
mi_local_global: 0.012192405760288239
mi_global_fixed: -0.00592326745390892
rec_loss: 0.8716923594474792
kld_loss: 0.06236515939235687
mi_local_global: 0.013679485768079758
mi_global_fixed: -0.005211349576711655
rec_loss: 0.8639009594917297
kld_loss: 0.05511922389268875
mi_local_global: 0.009854055941104889
mi_global_fixed: -0.006456360220909119
rec_loss: 0.8666983246803284
kld_loss: 0.0584426149725914
mi_local_global: 0.010690610855817795
mi_global_fixed: -0.004981037229299545
rec_loss: 0.8579800724983215
kld_loss: 0.035542845726013184
mi_local_global: 0.01119280606508255
mi_global_fixed: -0.008067663758993149
rec_loss: 0.8560543060302734
kld_loss: 0.052306968718767166
mi_local_global:

2023-01-07 21:16:44,617 (laplacian_with_domain_separation_1MINE_VAE_client:142)INFO: {'Role': 'Client #2', 'Round': 17, 'Results_raw': {'train_imp_ratio': -764.506218, 'train_avg_loss': 0.229094, 'train_loss': 117.296207, 'train_total': 512}}


mi_local_global: 0.013695426285266876
mi_global_fixed: -0.006743185222148895
rec_loss: 0.8639299273490906
kld_loss: 0.04883965849876404
mi_local_global: -0.0019693300127983093
mi_global_fixed: -0.009459134191274643
rec_loss: 0.9737675786018372
kld_loss: 0.37219053506851196
mi_local_global: -0.0009734369814395905
mi_global_fixed: -0.007982302457094193
rec_loss: 0.9777351021766663
kld_loss: 0.35624751448631287
mi_local_global: -0.0012807101011276245
mi_global_fixed: -0.00785057619214058
rec_loss: 0.9791156053543091
kld_loss: 0.3324655294418335
mi_local_global: -0.0005728043615818024
mi_global_fixed: -0.006974399089813232
rec_loss: 0.9741579294204712
kld_loss: 0.3787314295768738
mi_local_global: 0.0007171221077442169
mi_global_fixed: -0.009419895708560944
rec_loss: 0.9760619401931763
kld_loss: 0.3705745339393616
mi_local_global: -0.00019485130906105042
mi_global_fixed: -0.009593009948730469
rec_loss: 0.9774868488311768
kld_loss: 0.366199254989624
mi_local_global: 0.00039722397923469543
mi

KeyboardInterrupt: 

client 0 times: 0.346s
bei normal fedavg: 0.15s
mit MINE: 1.7 Sekunden


+

In [ ]:
777-431


In [ ]:
data

In [ ]:
import os

,,0After 10 rounds:

diff: 0, csd: 0:
server: -8
c1 t/val: 0.375 /0.37
c2 t/val: 0.678 /0.606
c3 t/val: 0.699 /0.608
c4 t/val: 0.599 /0.537

diff: 0, csd: 0:
server: -4.5
c1 t/val: 0.372879 /0.328
c2 t/val: 0.678992 /0.605809
c3 t/val: 0.70166 /0.608609
c4 t/val: 0.597455 /0.534253



diff: 10, csd: 0:
server: -35
c1 t/val: 0.482/0.52
c2 t/val: 0.694/0.65
c3 t/val: 0.724/0.65
c4 t/val: 0.674/0.575

diff: 1, csd: 0:
server: -9.6
c1 t/val: 0.378194/0.319349
c2 t/val: 0.67461/0.609163
c3 t/val: 0.705559/0.615019
c4 t/val: 0.605581/0.572086

diff: 0, csd: 1e4:
server: -18
c1 t/val: 0.280962/0.23074
c2 t/val: 0.682652/0.603175
c3 t/val: 0.699094/0.67027
c4 t/val: 0.594985/0.609611

diff: 0 csd: 1e8
server: -18
c1 t/val: 0.2606/0.348(0.197)
c2 t/val: 0.653/0.0.602
c3 t/val: 0.691/0.597
c4 t/val: 0.581/0.629

diff: 0 csd: 1e8
server: -12.7
c1 t/val: 0.262647/0.249875
c2 t/val: 0.649273/0.0.600435
c3 t/val: 0.696132/0.615158
c4 t/val: 0.586597/0.656992


diff: 0.1 csd: 0
server: -11
c1 t/val: 0.2606/0.332837
c2 t/val: 0.678326/0.0.610099
c3 t/val: 0.704083/0.616179
c4 t/val: 0.604177/0.559658




In [ ]:
os.environ['CUBLAS_WORKSPACE_CONFIG']